## Event Logs Analysis 

In [0]:
consumption_events_logs_df = (
    spark.read
         .format("delta")
         .load("/mnt/consumption/barstool/consumption_events_logs")
)

# quick peek
consumption_events_logs_df=consumption_events_logs_df.dropDuplicates()
display(consumption_events_logs_df.limit(20))

anon_id,content_id,content_title,content_type,duration_sec,share_url,TAGS,TALENT,FRANCHISE,published_at,talent_details,franchise_details,event_name,event_timestamp,event_type,event_date,event_month,event_year,event_hour,event_day,page_title,geo_area_code,geo_city,geo_country_code,geo_country_code3,geo_postal_code,geo_region,app_os,app_version,content_duration
8406dff613becd2ee3da6e0ef518992d,article-3544499,"The Knicks March Into Boston, Steal Game 1 From The Celtics, And Prove They Can Beat Those Fucking Guys",article,null,https://s.barstool.link/c/article-3544499,List(),List(94461631),UNKNOWN,2025-05-06T02:19:59.000+0000,"List(List(Clem, Clem))","List(List(null, null))",ContentStarted,2025-05-06T03:35:34.759Z,track,2025-05-06,5,2025,3,3,,703,ashburn,US,USA,20147,VA,iOS,60.0.3,null
2374cee2300df01a36ea1c559a578be0,article-3544433,Michigan Is Self-Imposing A Two Game Suspension (But NOT The First Two Games) To Lessen The Incoming Hammer From The NCAA,article,null,https://s.barstool.link/c/article-3544433,List(),List(81445381),UNKNOWN,2025-05-05T14:10:00.000+0000,"List(List(Ohios Tate, Ohios Tate))","List(List(null, null))",ContentStarted,2025-05-06T03:04:09.678Z,track,2025-05-06,5,2025,3,3,,703,ashburn,US,USA,20147,VA,iOS,60.0.3,null
4155e186e68eca717060d8592f11f52c,article-3544454,Why I’ll Take Real-Life Rejection Over Online Dating Any Day,article,null,https://s.barstool.link/c/article-3544454,List(),List(58276848),UNKNOWN,2025-05-05T18:00:00.000+0000,"List(List(Nicky Smokes, Nicky Smokes))","List(List(null, null))",null,2025-05-06T03:06:14.482Z,page,2025-05-06,5,2025,3,3,Why I’ll Take Real-Life Rejection Over Online Dating Any Day,813,tampa,US,USA,33624,FL,iOS,60.0.3,null
9b168b45e97777d2f1874785d223537d,vod-s9lJrF4ZopfXNXTybhC1ADIT,"Dave and Meek Preview The Bet Gala - Barstool Rundown - May 5th, 2025",vod,null,https://s.barstool.link/c/vod-s9lJrF4ZopfXNXTybhC1ADIT,List(),List(11236915),14,2025-05-05T21:30:00.000+0000,"List(List(Nate, Nate))","List(List(Barstool Rundown, Rundown))",ContentStarted,2025-05-06T03:04:36.658Z,track,2025-05-06,5,2025,3,3,,703,ashburn,US,USA,20147,VA,iOS,60.0.3,418.883803233
977db5714321325c8e1e408ff6ec6dea,viral-original-JRgIvi5Y3OPh0LChAZedyhsr,"Inside the World of Kidfluencers & Streamers (ft. Mook) | Apr 17, 2025",viral-original,null,https://s.barstool.link/c/viral-original-JRgIvi5Y3OPh0LChAZedyhsr,List(),List(),149,2025-04-17T04:00:00.000+0000,"List(List(null, null))","List(List(Macrodosing, null))",ContentStarted,2025-05-06T03:05:00.544Z,track,2025-05-06,5,2025,3,3,,703,ashburn,US,USA,20147,VA,iOS,60.0.3,69.664691638
438a87094b74b6ed8bef70ed7339e200,vod-UbKYmn7LNX2FiJkXUSujm5HM,Frank Walks Episode 53: Bartolo Colon presented by BODYARMOR,vod,null,https://s.barstool.link/c/vod-UbKYmn7LNX2FiJkXUSujm5HM,"List(sports, frank-the-tank, barstool-sports, barstool, frank-the-tank-walk, bartolo-colon, mlb, pitchers-home-runs, cy-young, los-angeles-angels, big-sexy)",List(1747017),UNKNOWN,2025-05-02T20:20:00.000+0000,"List(List(Frank The Tank, Frank The Tank))","List(List(null, null))",ContentStarted,2025-05-05T07:10:10.270Z,track,2025-05-05,5,2025,7,2,,703,ashburn,US,USA,20147,VA,iOS,60.0.3,772.148220398
4c22fa6319994a5ab4f9134d64c5fd05,article-3544483,"An Ex-CIA Scientist Tells Joe Rogan the US Has 'More Than 10' Recovered UFOs, Some of Which are 'Donations'",article,null,https://s.barstool.link/c/article-3544483,"List(ufos, uaps, joe-rogan, cia, remote-viewing)",List(49072379),UNKNOWN,2025-05-05T21:40:00.000+0000,"List(List(Jerry Thornton, Jerry Thornton))","List(List(null, null))",null,2025-05-06T02:35:31.257Z,page,2025-05-06,5,2025,2,3,"An Ex-CIA Scientist Tells Joe Rogan the US Has 'More Than 10' Recovered UFOs, Some of Which are 'Donations'",781,whitman,US,USA,02382,MA,iOS,60.0.3,null
57409d63280930bd9a7edceaf4f2130b,gallery-1173926,Barstool Tuesday Morning Smokeshow - Jocelyn @jocelyn_neill,gallery,null,https://s.barstool.link/c/gallery-1173926,"List(instagram, smokeshow, kmarko)",List(MUgEini

In [0]:
consumption_events_logs_df.createOrReplaceTempView("consumption_events_logs")

###  1. Content by Type Analysis
- considering event_type 'track' not 'page' (which means when media is palyed in the content).  Only `event_name = 'ContentStarted'` ➜ true signal that a user actively began consuming the content. 
- because we are filtering on  event_name = 'ContentStarted'.
- We ignore `event_type = 'page'` because it often carries null `event_name` and simply means a page view.  

view_count` – total “starts” per `CONTENT_TYPE`
  * `unique_contents` – distinct `content_id` per type (helps gauge catalogue size)

In [0]:
content_by_type = spark.sql("""
    SELECT 
        CONTENT_TYPE,
        COUNT(*) as view_count,
        COUNT(DISTINCT content_id) as unique_contents
    FROM consumption_events_logs
    WHERE event_name = 'ContentStarted'
    GROUP BY CONTENT_TYPE
    ORDER BY view_count DESC
""")
print("Content Views by Type:")

content_by_type.display()

Content Views by Type:


CONTENT_TYPE,view_count,unique_contents
article,37378,270
viral-original,7894,698
podcast-episode,5133,37
vod,2506,89
viral-ugc,989,61
gallery,77,33


Databricks visualization. Run in Databricks to view.

From the above dashboard, we can clearly see content_type - article has more views count. for this data 
- Articles dominates
- Podcasts have modest absolute views but the highest views‑per‑asset, hinting at a loyal audience worth nurturing.
- Gallery & viral‑ugc perform poorly

###  Platform usgae analysis
Knowing which operating system drives **ContentStarted** events helps product and ad‑tech teams prioritise feature work, SDK updates, and QA efforts.
But in this scenerio could be real we have data only for IOS assumueably picked from ios bucket)

In [0]:

platform_usage = spark.sql("""
    SELECT 
        app_os,
        COUNT(*) as view_count
    FROM consumption_events_logs
    WHERE event_name = 'ContentStarted'
    GROUP BY app_os
    ORDER BY view_count DESC
""")
print("Content Views by Platform:")
platform_usage.display()

Content Views by Platform:


Databricks visualization. Run in Databricks to view.

app_os,view_count
iOS,53977


### When Do Users Hit Play?  — Hourly Activity Curve

pinpointing the **hours with the most ContentStarted events** lets programming, social, and ad‑ops teams schedule releases and push notifications when users are most receptive.

In [0]:

hourly_usage = spark.sql("""
    SELECT 
        event_hour,
        COUNT(*) as view_count
    FROM consumption_events_logs
    WHERE event_name = 'ContentStarted'
    GROUP BY event_hour
    ORDER BY view_count
""")
print("Content Views by Hour of Day:")
hourly_usage.display()

Content Views by Hour of Day:


event_hour,view_count
7,309
8,470
6,582
5,699
9,708
4,1056
10,1394
11,1856
13,1912
3,1935


Databricks visualization. Run in Databricks to view.

 UTC Hour | View Count (≈) | EDT (UTC‑4) | Likely Behaviour |
|---------:|---------------:|------------:|------------------|
| **00‑02** | 3.6 k–3.4 k    | 8‑10 PM    | Prime‑time browse(post‑dinner sports recaps, live‑stream replays) |
| 03‑08     | < 1 k          | 11 PM‑4 AM | Overnight; most U.S. users offline |
| 09‑10     | 0.7 k–1.4 k    | 5‑6 AM     | Early risers|
| 11‑13     | 1.7 k–2.0 k    | 7‑9 AM     | Morning catch‑up, quick reads before work |
| **14‑18** | 2.5 k–3.2 k    | 10 AM‑2 PM | Lunch‑hour surge; good for short‑form or podcasts |
| 19‑23     | 2.8 k–3.4 k    | 3‑7 PM     | Second incline heading into East‑Coast evening |

### Daily Active Users (DAU)
ContentStarted = “real engagement”, not passive page views.



In [0]:

#Dialy active users
dialy_active_users = spark.sql("""
SELECT
    event_date,
    COUNT(DISTINCT anon_id) AS dau
FROM consumption_events_logs
WHERE event_name = 'ContentStarted'
GROUP BY event_date
ORDER BY event_date;"""
)
dialy_active_users.display()

event_date,dau
2025-05-05,15843
2025-05-06,7535


Databricks visualization. Run in Databricks to view.

- One‑day drop of ~9 k users. 
- Weekend vs. weekday effect (May 4 was a Sunday).
- Partial data extracted
- Live event on the 4th boosting traffic.


### Leaderboard of engagement 
- **Shows which articles/videos accumulated the most watch time, not just clicks.**
- **Helps editorial decide whether long‑form pieces (higher minutes) outperform quick hits.**

In [0]:
spark.sql("""
SELECT
    content_id,
    FIRST(content_title)          AS content_title,   
    FIRST(content_type)           AS content_type,
    ROUND(SUM(duration_sec)/60,1) AS minutes_watched
FROM consumption_events_logs
WHERE event_name = 'ContentStarted'        
GROUP BY content_id
ORDER BY minutes_watched DESC
LIMIT 10;""").display()



content_id,content_title,content_type,minutes_watched
podcast-episode-4CBIqI133ODlkCEOYTImDINc,"Ryan Whitney Talking Stanley Cup Playoffs, Warriors And Nuggets Advance, Kentucky Derby + Jordon Update Of The Week",podcast-episode,85770.3
podcast-episode-DaU6Srr0unkS95JyvNtXr9Pd,Spittin’ Chiclets Episode 560: Featuring Bob Stauffer & Gary Lawless,podcast-episode,21008.5
podcast-episode-Tiu474zZunXBfkpXsKuxoarf,An In-Depth Look Into the Delco Pooper with Mike Rainey | The Yak 5-5-25,podcast-episode,19107.3
podcast-episode-WQAAHuTzh9hnELvH6eg2HQ6J,Kate Reveals Her List of Things That Are Back After Her Girls Trip to Mexico | The Yak 4-28-25,podcast-episode,10815.2
podcast-episode-Q09J1OVwGKZm9NBpJdTaPpfs,Big Cat Signs Off on Barstool Australia | Drop A Pin Ep. 30,podcast-episode,8384.7
podcast-episode-PP465qAuWXo8nBKDdMNWF61l,"Randy Moss Talking Kentucky Derby, Joakim Noah In Studio, The Lakers Go Out Sad, Jalen Brunson Ends The Pistons, Hockey Playoffs + Fyre Fest of The Week",podcast-episode,7614.5
podcast-episode-RlEevPYcJRUQH99nuq4wg4Xh,"NFL Draft With Peter Schrager, Stavros Halkias In Studio, NFL Draft Recap, Mel Kiper's meltdown, NBA/NHL Playoffs And Bill Belichick Might Be In A Conservatorship",podcast-episode,6606.5
podcast-episode-lbrea5VqH8xit03X0GBddVgl,"Dave Portnoy Announces Golf Creator Tournament ""The Internet Invitational"" | The Unnamed Show - Episode 56",podcast-episode,4171.5
vod-s9lJrF4ZopfXNXTybhC1ADIT,"Dave and Meek Preview The Bet Gala - Barstool Rundown - May 5th, 2025",vod,3449.5
podcast-episode-PkpjSCuXZNhlHuNltHnAb0sy,"Dave Portnoy and Meek Phil Preview The Bet Gala - May 5th, 2025",podcast-episode,2483.8


Databricks visualization. Run in Databricks to view.

###  Dialy Active Users by Country (all‑time sample)
- U.S. dominates the current sample; Canada is a distant second.

In [0]:
%sql
-- Unique users per country (all time)
SELECT
    geo_country_code3        AS country,
    COUNT(DISTINCT anon_id) AS users
FROM consumption_events_logs
GROUP BY geo_country_code3
ORDER BY users DESC;

country,users
USA,25537
CAN,367
GBR,49
MEX,17
DEU,14
AUS,13
ESP,9
ITA,8
NLD,7
PRI,7


Databricks visualization. Run in Databricks to view.